In [1]:
# https://vincentteyssier.medium.com/filtering-csv-files-bigger-than-memory-to-a-pandas-dataframe-3ab51ff993fd

In [1]:
import psutil
import pandas as pd
import csv
svmem = psutil.virtual_memory()
print(svmem.available)

8564895744


In [2]:
PATH = r"c:\\Users\\alexe\\ipynb\\ipynb10\\magni_all_row.csv"
df_sample = pd.read_csv(PATH, nrows=10)
df_sample_size = df_sample.memory_usage(index=True).sum()
print(df_sample_size)
print(df_sample)

1728
   Unnamed: 0                                 ДокументПолученный  \
0           0  Документ полученный CNTM-01072227 от 05.05.202...   
1           1  Документ полученный CLCM-00078534 от 31.05.202...   
2           2  Документ полученный CLCM-00078562 от 31.05.202...   
3           3  Документ полученный NNCM-00080965 от 31.05.202...   
4           4  Документ полученный CLCM-00082021 от 31.05.202...   
5           5  Документ полученный SPCM-00045591 от 31.05.202...   
6           6  Документ полученный SPCM-00045851 от 31.05.202...   
7           7  Документ полученный SPCM-00045851 от 31.05.202...   
8           8  Документ полученный SPCM-00045852 от 31.05.202...   
9           9  Документ полученный SPCM-00045852 от 31.05.202...   

  НоменклатураКод                                       Номенклатура  \
0      RCC-000592  Транспортировка товара от поставщика (по терри...   
1      CNT-001254                  Аренда технологического помещения   
2      CNT-001254             

In [3]:

# define a chunksize that would occupy a maximum of 1Gb
# we divide by 10 because we have selected 10 lines in our df_sample
my_chunk = (1000000000/df_sample_size)/10
my_chunk = int(my_chunk//1) # we get the integer part
print(my_chunk)
# create the iterator


57870


In [4]:
# df.apply(lambda row: row.astype(str).str.contains('НЕ ВЫБРАН'.lower(), case=False).any(), axis=1)
# df.apply(lambda row: row.astype(str).str.contains('<NULL>'.lower(), case=False).any(), axis=1)

In [8]:
# !!!!!!!!!!!!!!!! good!!!!!!!!!

iter_csv = pd.read_csv(
    PATH,
    iterator=True,
    chunksize=my_chunk, low_memory=False)  # low_memory=False
# concatenate according to a filter to our result dataframe
df_result_nv = pd.concat(
    [chunk[chunk['Модель']=='НЕ ВЫБРАН'] for chunk in iter_csv])
#     [chunk.apply(lambda row: row.astype(str).str.contains('НЕ ВЫБРАН').any(), axis=1)   

df_result_nv.sample(5)

,Unnamed: 0,ДокументПолученный,НоменклатураКод,Номенклатура,МодельКод,Модель,Количество,ВалютаДокумента,Сумма,СтавкаНДС,СуммаНДС,СуммаСНДС,СуммаДокумента,КонтрагентИНН,Контрагент,Договор,ПодразделениеКод,Подразделение,ДатаОтгрузки,Цена
14450542,602128,Документ полученный GKCM-00056089 от 27.04.202...,CNT-001337,НЕ ВЫБРАН,CNT-000865,НЕ ВЫБРАН,"2 148,000",руб.,NaN,20%,NaN,NaN,NaN,7701010842.0,ЕДИНАЯ ЕВРОПА-ХОЛДИНГ ОАО,Договор № ГК/3/3211/11 от 01.10.2011,GKC-RC7122,Склад РМ МХ Тула,27.04.2021 0:00:00,NaN
2035836,379795,Документ полученный GKCM-00063433 от 07.06.202...,CNT-001337,НЕ ВЫБРАН,CNT-000865,НЕ ВЫБРАН,NaN,руб.,NaN,20%,NaN,NaN,"1 344 198,24",7816520261.0,Техноинформ-Балтика ООО,Договор № РЦЦ/55210/19 от 13.09.2019,GKC-RC2622,Склад РМ МХ Лермонтово,11.06.2021 0:00:00,NaN
24096535,599773,Документ полученный GKCM-00024578 от 05.10.202...,CNT-001337,НЕ ВЫБРАН,CNT-000865,НЕ ВЫБРАН,"24,000",руб.,NaN,20%,NaN,NaN,NaN,7729265128.0,Алькор и Ко ООО,Договор № ГК/52000/16 от 01.07.2016,GKC-RC3722,Склад РМ МХ Иваново,07.10.2020 0:00:00,NaN
14450985,602571,Документ полученный GKCM-00057733 от 29.04.202...,CNT-001337,НЕ ВЫБРАН,CNT-000865,НЕ ВЫБРАН,NaN,руб.,NaN,20%,NaN,NaN,"480 070,80",7816520261.0,Техноинформ-Балтика ООО,Договор № РЦЦ/55210/19 от 13.09.2019,GKC-RC5522,Склад РМ МХ Омск,07.05.2021 0:00:00,NaN
9538923,569727,Документ полученный GKC-857704 от 05.03.2020 0...,CNT-001337,НЕ ВЫБРАН,CNT-000865,НЕ ВЫБРАН,"970,000",руб.,NaN,20%,NaN,NaN,NaN,1659092774.0,Парли ООО,Договор № ГК/16337/17 от 14.03.2017,GKC-RC3722,Склад РМ МХ Иваново,07.03.2020 0:00:00,NaN


DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\alexe\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\alexe\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):

In [5]:
iter_csv = pd.read_csv(
    PATH,
    iterator=True,
    chunksize=my_chunk)  # low_memory=False
# concatenate according to a filter to our result dataframe
df_result = pd.concat(
    [chunk[chunk['Сумма']=='<NULL>']
    for chunk in iter_csv])
df_result.sample(5)

C:\Users\alexe\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\alexe\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\alexe\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,Unnamed: 0,ДокументПолученный,НоменклатураКод,Номенклатура,МодельКод,Модель,Количество,ВалютаДокумента,Сумма,СтавкаНДС,СуммаНДС,СуммаСНДС,СуммаДокумента,КонтрагентИНН,Контрагент,Договор,ПодразделениеКод,Подразделение,ДатаОтгрузки,Цена
28690566,1391289,Документ полученный GKCM-00035731 от 15.12.202...,<NULL>,<NULL>,<NULL>,<NULL>,<NULL>,руб.,<NULL>,<NULL>,<NULL>,<NULL>,NaN,5037050871.0,Артпласт АО,Договор № ГК/4/4132/10 от 01.09.2010,<NULL>,<NULL>,17.12.2020 0:00:00,<NULL>
3882532,352757,Документ полученный KRC-137290 от 31.01.2020 0...,<NULL>,<NULL>,<NULL>,<NULL>,<NULL>,руб.,<NULL>,<NULL>,<NULL>,<NULL>,NaN,232010208550.0,Емцев Дмитрий Александрович ИП,Договор № СчФ/71363/18 от 21.12.2018,<NULL>,<NULL>,31.01.2020 0:00:00,<NULL>
4079902,550127,Документ полученный GKC-852129 от 24.01.2020 0...,<NULL>,<NULL>,<NULL>,<NULL>,<NULL>,руб.,<NULL>,<NULL>,<NULL>,<NULL>,NaN,7705407550.0,Паблик Тотем ООО,Договор № ГК/81836/16 от 30.11.2016,<NULL>,<NULL>,31.01.2020 0:00:00,<NULL>
28236650,937373,Документ полученный NNCM-00068397 от 31.12.202...,<NULL>,<NULL>,<NULL>,<NULL>,<NULL>,руб.,<NULL>,<NULL>,<NULL>,<NULL>,NaN,3702733438.0,ИвГТЭ АО,Договор № б/н от 01.07.2019,<NULL>,<NULL>,31.12.2020 0:00:00,<NULL>
28356878,1057601,Документ полученный NNCM-00068292 от 31.12.202...,<NULL>,<NULL>,<NULL>,<NULL>,<NULL>,руб.,<NULL>,<NULL>,<NULL>,<NULL>,NaN,3702733438,ИвГТЭ АО,Договор № б/н от 01.07.2019,<NULL>,<NULL>,31.12.2020 0:00:00,<NULL>


In [6]:
# всего строк с "НЕ ВЫБРАН"   21337    с '<NULL>'    6735  - Это R считал   !!!!
df_result.shape

(31, 20)

In [7]:
df_result.to_csv(r'df_result_NULL.csv', encoding='utf-8', header='true')

In [8]:
iter_csv = pd.read_csv(
    PATH,
    iterator=True,
    chunksize=my_chunk, low_memory=False)  # low_memory=False
# concatenate according to a filter to our result dataframe
df_result_nv = pd.concat(
    [chunk[chunk['Модель']=='НЕ ВЫБРАН']
    for chunk in iter_csv])
# df_result_nv.sample(5)

# chunk.apply(lambda row: row.astype(str).str.contains('НЕ ВЫБРАН').any(), axis=1)

ValueError: a must be greater than 0 unless no samples are taken

In [7]:
def search(dataFrame, item):
    mask = (dataFrame.applymap(lambda x: isinstance(x, str) and item in x)).any(1)
    return dataFrame[mask]

In [9]:
df_result_nv.shape

(21134, 20)

In [10]:
iter_csv = pd.read_csv(
    PATH,
    iterator=True,
    chunksize=my_chunk, low_memory=False)  # low_memory=False
# concatenate according to a filter to our result dataframe
df_result_nv = pd.concat(
    [chunk[chunk['Контрагент'].notna()]
    for chunk in iter_csv])
df_result_nv.sample(5)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [14]:
df_result_nv.to_csv(r'df_result_dont_select.csv', encoding='utf-8', header='true')

In [15]:
df_result_nv.shape # (21134, 20)

(21134, 20)

In [ ]:
import psutil
import pandas as pd
import csv
svmem = psutil.virtual_memory()
print (svmem.available)
PATH = r”C:\tmp\dataset\tf_train.csv”
df_sample = pd.read_csv(PATH, nrows=10)
df_sample_size = df_sample.memory_usage(index=True).sum()
print (df_sample_size)
print (df_sample)
# define a chunksize that would occupy a maximum of 1Gb
# we divide by 10 because we have selected 10 lines in our df_sample
my_chunk = (1000000000 / df_sample_size)/10
my_chunk = int(my_chunk//1) # we get the integer part
print (my_chunk)
# create the iterator
iter_csv = pd.read_csv(
    PATH,
    iterator=True,
    chunksize=my_chunk)
# concatenate according to a filter to our result dataframe
df_result = pd.concat(
    [chunk[chunk[‘n3’]>0]
    for chunk in iter_csv])
print (df_result)